In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import backend as K

from preprocessing import preprocesamiento_basico
from graficos_modelos import mostrar_reporte_clasificacion, graficar_auc_roc, graficar_matriz_confusion, graficar_curva_aprendizaje
from funciones_auxiliares import traer_datasets, traer_dataset_prediccion_final, separar_dataset, encontrar_hiperparametros_GSCV, mapear_target_binario

# Redes neuronales

Definimos el método que generará la red neuronal con una arquitectura constante, la cual tendra dos capas ocultas de 10 y 8 neuronas respectivamente. A su vez, se utilizara la funcion de activación de tangente hiperbólica en las capas ocultas, y una sigmoidea en la capa de output ya que se trata de un problema de clasificación binaria. Por el mismo motivo, la funcion de perdida a optimizar será la cross-entropy.
El método de regularización a utilizar sera dropout con parametro 1/3 para mantener el modelo lo más simple posible y no sobrecargarlo con computo.

In [2]:
def crear_red(optimizer='SGD'):
    red_neuronal = Sequential()
    #, kernel_regularizer=l2(0.001)
    red_neuronal.add(Dense(8, input_dim=18, activation='tanh'))
    red_neuronal.add(Dropout(0.25))
    red_neuronal.add(Dense(5, activation='tanh'))
    red_neuronal.add(Dropout(0.25))
    red_neuronal.add(Dense(1, activation='sigmoid'))
    red_neuronal.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=[keras.metrics.AUC()])
    return red_neuronal

## Modelo con preprocesamiento básico:

Obtención de datos y preprocesamiento

In [3]:
df, df_sin_target, solo_target = traer_datasets()

X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)

X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_test.is_copy=False

y_train.set_index('id', inplace=True)
y_train = y_train.sort_values(by=['id'], ascending=True).copy()

y_test.set_index('id', inplace=True)
y_test = y_test.sort_values(by=['id'], ascending=True).copy()

In [ ]:
dataframes_procesados = preprocesamiento_basico([X_train, X_test])
X_train = dataframes_procesados[0]
X_test = dataframes_procesados[1]

Planteamos como posibles hiperpametros dos optimizadores, SGD que resulta el más simple de los posibles, y adam que es un optimizador de complejidad media. Como el problema a resolver no es extremadamente complejo, no hay necesidad de complejizar tanto el computo de la red con optimizadores mas elaborados para obtener una buena performance.
Los learning rates propuestos se consideran relativamente valores bajos para que el aprendizaje sea suave y no abrupto/errático

Siendo los mejores hiperparametros:

In [ ]:
mapeo_binario_v = np.vectorize(mapear_target_binario)

y_train_np = y_train['llovieron_hamburguesas_al_dia_siguiente'].to_numpy()
y_train_binario = mapeo_binario_v(y_train_np)

y_test_np = y_test['llovieron_hamburguesas_al_dia_siguiente'].to_numpy()
y_test_binario = mapeo_binario_v(y_test_np)


Creamos entonces la red con el mejor optimizador, y entrenamos para la mejor cantidad de epochs halladas

In [ ]:
red_n = crear_red(keras.optimizers.SGD(learning_rate=0.0001))
red_n.summary()

In [ ]:
historia = red_n.fit(
    X_train.values, y_train_binario, epochs=100, 
    validation_data=(X_test.values, y_test_binario), 
    verbose=1,
)

### Curva de aprendizaje


Graficamos la curva de aprendizaje para tener una mejor idea de como fue el entrenamiento de nuestra red

In [ ]:
graficar_curva_aprendizaje(historia,'loss')
graficar_curva_aprendizaje(historia,'auc')

### Predicción del modelo sobre holdout

Una vez que vimos que no se produjo un overfitting y que la métrica alcanzada es lo suficientemente buena, realizamos la predicción sobre los valores de holdout, y gráficamos la curva AUC-ROC y la matriz de confusión

In [ ]:
y_pred = red_n.predict(X_test)

In [ ]:
graficar_auc_roc(y_test_binario, y_pred) 

In [ ]:
graficar_matriz_confusion(y_test_binario, y_pred.round())

Sin embargo vemos que nuestra red no es capaz de detectar correctamente los true positives, ya que en la matriz de confusion hay 0 de dichos casos acertados. 
Intentaremos mejorar dicha performance duplicando la cantidad de epochs y utilizando una red mas simple

In [ ]:
def crear_red_simplificada(optimizer='adam'):
    red_neuronal = Sequential()
    red_neuronal.add(Dense(5, input_dim=18, activation='tanh'))
    red_neuronal.add(Dropout(0.25))
    red_neuronal.add(Dense(3, activation='tanh', kernel_regularizer=l2(0.001)))
    red_neuronal.add(Dropout(0.25))
    red_neuronal.add(Dense(1, activation='sigmoid', kernel_regularizer=l2(0.001)))
    red_neuronal.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=[keras.metrics.AUC()])
    return red_neuronal

In [ ]:
red_n = crear_red_simplificada(keras.optimizers.SGD(learning_rate=0.0001))
red_n.summary()

In [ ]:
historia = red_n.fit(
    X_train.values, y_train_binario, epochs=200, 
    validation_data=(X_test.values, y_test_binario), 
    verbose=1,
)

In [ ]:
graficar_curva_aprendizaje(historia,'loss')
graficar_curva_aprendizaje(historia,'auc_1')


In [ ]:
y_pred = red_n.predict(X_test)

In [ ]:
graficar_auc_roc(y_test_binario, y_pred) 

In [ ]:
graficar_matriz_confusion(y_test_binario, y_pred.round())